In [3]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
print(t.fit_on_texts(docs))
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

None
[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [2]:
print(t.fit_on_texts(docs))

None


In [10]:
!pip install plotly
!pip install nltk

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 1.4MB 1.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /Users/OFlynn/Library/Caches/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built nltk
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

In [19]:
# read the entire file into a python array
with open('review.json', 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
data_df = pd.read_json(data_json_str)
print(data_df.head())
# df= df.dropna()
# df = df[df.stars.apply(lambda x: x.isnumeric())]
# df = df[df.stars.apply(lambda x: x !="")]
# df = df[df.text.apply(lambda x: x !="")]

              business_id  cool       date  funny               review_id  \
0  0W4lkclzZThpx3V65bVgig     0 2016-05-28      0  v0i_UHJMo_hPBq9bxWvW4w   
1  AEx2SYEUJmTxVVB18LlCwA     0 2016-05-28      0  vkVSCC7xljjrAI4UGfnKEQ   
2  VR6GpWIda3SfvPC-lg9H3w     0 2016-05-28      0  n6QzIUObkYshz4dz2QRJTw   
3  CKC0-MOWMqoeWf6s-szl8g     0 2016-05-28      0  MV3CcKScW05u5LVfF6ok0g   
4  ACFtxLv8pGrrxMm6EgjreA     0 2016-05-28      0  IXvOzsEMYtiJI0CARmj77Q   

   stars                                               text  useful  \
0      5  Love the staff, love the meat, love the place....       0   
1      5  Super simple place but amazing nonetheless. It...       0   
2      5  Small unassuming place that changes their menu...       0   
3      5  Lester's is located in a beautiful neighborhoo...       0   
4      4  Love coming here. Yes the place always needs t...       0   

                  user_id  
0  bv2nCi5Qv5vroFiqKGopiw  
1  bv2nCi5Qv5vroFiqKGopiw  
2  bv2nCi5Qv5vroFiqKGopiw 

In [22]:
data_df_short = data_df[:10000]
data_df_short.to_json('reviews_short.json')

In [25]:
df= data_df_short.dropna()
# df = data_df_short[data_df_short.stars.apply(lambda x: x.isdigit())]
df = data_df_short[data_df_short.stars.apply(lambda x: x !="")]
df = data_df_short[data_df_short.text.apply(lambda x: x !="")]

In [26]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

In [34]:
### Text Normalizing function. Part of the following function was taken from this link. 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk, re
nltk.download('stopwords')

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text
## apply the above function to df['text']
df['text'] = df['text'].map(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/OFlynn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [71]:
## Network architecture
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, np.array(labels), validation_split=0.4, epochs=3)

Train on 6000 samples, validate on 4000 samples
Epoch 1/3
6000/6000 [==============================] - 15s 3ms/step - loss: 0.5621 - acc: 0.7113 - val_loss: 0.4726 - val_acc: 0.7915
Epoch 2/3
6000/6000 [==============================] - 14s 2ms/step - loss: 0.3479 - acc: 0.8563 - val_loss: 0.4339 - val_acc: 0.8063
Epoch 3/3
6000/6000 [==============================] - 14s 2ms/step - loss: 0.2287 - acc: 0.9137 - val_loss: 0.4884 - val_acc: 0.7937


In [37]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [67]:
test_comment = 'This is my favourite exhibition of all time!! I love the way that the curator placed all the paintings in the correct spots in the room'
test_comment2 = "not a Michelin starred restaurant, but dear me, I also think that you can expect some sort of standard when you go out to eat. Unfortunately, the standard at this restaurant was very poor. Where do I start? Having entered the restaurant, we had to wait for several minutes to be greeted and seated, when finally a blonde waitress said hi to us, or should i say to my husband only, in a very unproffesional flirty manner. She took us to our table, which was a very small square thing, with not much space for anything, and only about 20 cm away from another table, so actually we might as well join the tables together if we go next time (i don't think so !). The menu cards very ABSOLUTELY disgusting, covered in oil and all sorts of dirst with the foil peeling away from the paper . We ordered our drinks, and soon enough the 'over enthusiastic' waitress came back to take our orders , nearly siting down on my husbands lap AND I AM NOT JOKING ! The bruschetta I had as my starter, was nothing more than two slices of burnt Tesco's value garlic bread with a bit of tomato on top, which cost me nearly a fiver ..."
test_comment3 = "What will the future look like? After visiting the V&A’s latest exhibition, The Future Starts Here, which sets out to explore how design is “shaping the world of tomorrow”, I can tell you: sinister, that’s what. The show packs in 112 objects, products and “projects”, including solar-powered shirts that can charge a smartphone, and driverless cars – which, by my reckoning, have been a staple of crystal-ball-gazing exhibitions for years, even though engineers have yet to guarantee their safety."
test_comment4 = "A few years ago, at the J Paul Getty Museum that perches like a city state above a Los Angeles freeway, I asked for directions to the ticket office. There was no admission fee, explained a staffer. “The museum,” she added reverentially, “is Mr Getty’s gift to the world.” This gift was from the same oil squillionaire who, at his sprawling manor in Surrey, installed a guest payphone. Complicated guy. The whole family is quite complicated. I spent much of The Gettys: The World's Richest Arts Dynasty (BBC Two) trying to work out who was descended from whom. But then so did the Gettys. A Getty son once visited the office of his old man, who casually introduced him to a younger brother he’d never met. This documentary was only in passing about complex family dynamics. The main event was the Gettys’ cultural hegemony, and it felt like being hit over the head by a lazy press release. The family, blurbed the voiceover, “used the money to change the art world and British culture forever”. The acquisition of fading film stock “would change the course of British film history”. Getty was described as “the legendary”. So was William Randolph Hearst."

#3 is from https://www.telegraph.co.uk/tv/2018/04/14/gettys-worlds-richest-arts-dynasty-review-like-hit-head-lazy/
#4 is from https://www.telegraph.co.uk/art/reviews/future-starts-va-review-sinister-sobering-picture-tomorrows/

cleaned_comment = clean_text(test_comment)
comment_to_seq = tokenizer.texts_to_sequences([cleaned_comment])

cleaned_comment_2 = clean_text(test_comment2)
comment_to_seq_2 = tokenizer.texts_to_sequences([cleaned_comment_2])

cleaned_comment_3 = clean_text(test_comment3)
print(cleaned_comment_3)
comment_to_seq_3 = tokenizer.texts_to_sequences([cleaned_comment_3])

input_try = pad_sequences(comment_to_seq_3, maxlen=20)
print(input_try)
preds = model.predict(input_try)
print(preds)

futur look like visit v a s latest exhibit futur start here set explor design shape world tomorrow tell you : sinist that s what show pack 112 object product project includ solar - power shirt charg smartphon driverless car which reckon stapl crystal - ball - gaze exhibit year even though engin yet guarante safeti
[[ 9429  1606  1811   438 12089   185   556  8865  1595  2717   826  5173
   2398   128    24   104  2919   416  2112  3670]]
[[0.7231244]]
